In [9]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy import inspect
import requests
import hvplot.pandas
import holoviews as hv
from holoviews import dim
import matplotlib as mpl


In [10]:
load_dotenv()

database_connection_string = os.getenv("DATABASE_URI")

database_schema = os.getenv("DATABASE_SCHEMA")

engine = create_engine(database_connection_string, echo=False)


In [11]:
sql_query = """
SELECT t.contract_id,
       c.name as collection_name,
       c.address,
       tok.token_id,
       tok.id_num,
       tok.name as token_name,
       tok.rarity_score,
       tok.ranking,
       ct.average_token_rarity_score_for_collection,
       MIN(t.avg_price) as avg_price_for_collection,
       MIN(t.min_price) as min_price_for_collection,
       MAX(t.max_price) as max_price_for_collection,
       SUM(t.volume) as total_volume_for_collection,
       SUM(t.num_trades) as total_num_trades_for_collection,
       SUM(t.unique_buyers) as total_unique_buyers_for_collection
FROM network n
INNER JOIN collection c ON c.network_id = n.network_id
INNER JOIN (SELECT contract_id, ROUND(AVG(rarity_score), 2) AS average_token_rarity_score_for_collection FROM token GROUP BY contract_id) ct ON ct.contract_id = c.contract_id
INNER JOIN token tok ON tok.contract_id = ct.contract_id
INNER JOIN trade t ON t.contract_id = c.contract_id
WHERE n.network_id = 'ethereum' 
AND c.name IN ('CryptoPunks', 'BoredApeYachtClub', 'MutantApeYachtClub', 'Otherdeed', 'Azuki', 'CloneX', 'Moonbirds', 'Doodles', 'Meebits', 'Cool Cats', 'BoredApeKennelClub')
AND tok.ranking = 1
GROUP BY t.contract_id, c.name, c.address, tok.token_id, tok.id_num, tok.name, tok.rarity_score, tok.ranking, ct.average_token_rarity_score_for_collection
HAVING MIN(t.avg_price) > 0.0
ORDER BY SUM(t.volume)  DESC
"""
df = pd.read_sql_query(sql_query, con = engine)

display(df.head(10))
display(df.tail(10))


,contract_id,collection_name,address,token_id,id_num,token_name,rarity_score,ranking,average_token_rarity_score_for_collection,avg_price_for_collection,min_price_for_collection,max_price_for_collection,total_volume_for_collection,total_num_trades_for_collection,total_unique_buyers_for_collection
0,ethereum:7bd29408f11d2bfc23c34f18275bbf23bb716bc7,Meebits,7bd29408f11d2bfc23c34f18275bbf23bb716bc7,ethereum:7bd29408f11d2bfc23c34f18275bbf23bb716...,2a40,Meebit #10816,274.22,1,198.28,0.86,0.0,15000.00,3296996,42896,21715
1,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193...,1553,CryptoPunk #5459,86.19,1,58.35,0.05,0.0,124457.07,1003179,20775,14196
2,ethereum:bc4ca0eda7647a8ab7c2061c2e118a18a936f13d,BoredApeYachtClub,bc4ca0eda7647a8ab7c2061c2e118a18a936f13d,ethereum:bc4ca0eda7647a8ab7c2061c2e118a18a936f...,098b,,68.92,1,41.67,0.08,0.0,1024.00,844636,31680,22605
3,ethereum:60e4d786628fea6478f785a6d7e704777c86a7c6,MutantApeYachtClub,60e4d786628fea6478f785a6d7e704777c86a7c6,ethereum:60e4d786628fea6478f785a6d7e704777c86a...,702e,,54.58,1,22.62,0.51,0.0,800.00,581931,44008,34067
4,ethereum:34d85c9cdeb23fa97cb08333b511ac86e1c4e258,Otherdeed,34d85c9cdeb23fa97cb08333b511ac86e1c4e258,ethereum:34d85c9cdeb23fa97cb08333b511ac86e1c4e...,22e9,,44.76,1,34.06,1.38,0.0,851.54,483858,61503,33797
5,ethereum:ed5af388653567af2f388e6224dc7c4b3241c544,Azuki,ed5af388653567af2f388e6224dc7c4b3241c544,ethereum:ed5af388653567af2f388e6224dc7c4b3241c...,74,Azuki #116,142.92,1,114.00,1.59,0.0,459.06,345778,35829,25525
6,ethereum:49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,CloneX,49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,ethereum:49cf6f5d44e70224e2e23fdcdd2c053f30ada...,4b84,CloneX #18156,76.83,1,46.43,1.24,0.0,450.00,259070,20684,16438
7,ethereum:23581767a106ae21c074b2276d25e5c3e136a68b,Moonbirds,23581767a106ae21c074b2276d25e5c3e136a68b,ethereum:23581767a106ae21c074b2276d25e5c3e136a...,26ad,#9901,138.69,1,100.94,4.98,0.0,749.31,234913,18882,15435
8,ethereum:8a90cab2b38dba80c64b7734e58ee1db38b8992e,Doodles,8a90cab2b38dba80c64b7734e58ee1db38b8992e,ethereum:8a90cab2b38dba80c64b7734e58ee1db38b89...,0fd4,Doodle #4052,41.22,1,27.12,0.85,0.0,384.03,185186,27660,21312
9,ethereum:ba30e5f9bb24caa003e9f2f0497ad287fdf95623,BoredApeKennelClub,ba30e5f9bb24caa003e9f2f0497ad287fdf95623,ethereum:ba30e5f9bb24caa003e9f2f0497ad287fdf95...,08ef,,91.06,1,53.26,0.61,0.0,626.00,149124,25164,18551


,contract_id,collection_name,address,token_id,id_num,token_name,rarity_score,ranking,average_token_rarity_score_for_collection,avg_price_for_collection,min_price_for_collection,max_price_for_collection,total_volume_for_collection,total_num_trades_for_collection,total_unique_buyers_for_collection
1,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193...,1553,CryptoPunk #5459,86.19,1,58.35,0.05,0.0,124457.07,1003179,20775,14196
2,ethereum:bc4ca0eda7647a8ab7c2061c2e118a18a936f13d,BoredApeYachtClub,bc4ca0eda7647a8ab7c2061c2e118a18a936f13d,ethereum:bc4ca0eda7647a8ab7c2061c2e118a18a936f...,098b,,68.92,1,41.67,0.08,0.0,1024.00,844636,31680,22605
3,ethereum:60e4d786628fea6478f785a6d7e704777c86a7c6,MutantApeYachtClub,60e4d786628fea6478f785a6d7e704777c86a7c6,ethereum:60e4d786628fea6478f785a6d7e704777c86a...,702e,,54.58,1,22.62,0.51,0.0,800.00,581931,44008,34067
4,ethereum:34d85c9cdeb23fa97cb08333b511ac86e1c4e258,Otherdeed,34d85c9cdeb23fa97cb08333b511ac86e1c4e258,ethereum:34d85c9cdeb23fa97cb08333b511ac86e1c4e...,22e9,,44.76,1,34.06,1.38,0.0,851.54,483858,61503,33797
5,ethereum:ed5af388653567af2f388e6224dc7c4b3241c544,Azuki,ed5af388653567af2f388e6224dc7c4b3241c544,ethereum:ed5af388653567af2f388e6224dc7c4b3241c...,74,Azuki #116,142.92,1,114.00,1.59,0.0,459.06,345778,35829,25525
6,ethereum:49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,CloneX,49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,ethereum:49cf6f5d44e70224e2e23fdcdd2c053f30ada...,4b84,CloneX #18156,76.83,1,46.43,1.24,0.0,450.00,259070,20684,16438
7,ethereum:23581767a106ae21c074b2276d25e5c3e136a68b,Moonbirds,23581767a106ae21c074b2276d25e5c3e136a68b,ethereum:23581767a106ae21c074b2276d25e5c3e136a...,26ad,#9901,138.69,1,100.94,4.98,0.0,749.31,234913,18882,15435
8,ethereum:8a90cab2b38dba80c64b7734e58ee1db38b8992e,Doodles,8a90cab2b38dba80c64b7734e58ee1db38b8992e,ethereum:8a90cab2b38dba80c64b7734e58ee1db38b89...,0fd4,Doodle #4052,41.22,1,27.12,0.85,0.0,384.03,185186,27660,21312
9,ethereum:ba30e5f9bb24caa003e9f2f0497ad287fdf95623,BoredApeKennelClub,ba30e5f9bb24caa003e9f2f0497ad287fdf95623,ethereum:ba30e5f9bb24caa003e9f2f0497ad287fdf95...,08ef,,91.06,1,53.26,0.61,0.0,626.00,149124,25164,18551
10,ethereum:1a92f7381b9f03921564a437210bb9396471050c,Cool Cats,1a92f7381b9f03921564a437210bb9396471050c,ethereum:1a92f7381b9f03921564a437210bb93964710...,1f92,Cool Cat #8082,133.18,1,119.77,0.07,0.0,320.00,121496,32726,21411


In [12]:
collection_max_price_df = df.drop(columns=['contract_id', 'address', 'token_id', 'id_num', 'rarity_score', 'token_name', 'ranking', 'average_token_rarity_score_for_collection', 'avg_price_for_collection', 'min_price_for_collection', 'total_volume_for_collection', 'total_num_trades_for_collection', 'total_unique_buyers_for_collection'])

# convert USD to ETH to make overlay plot 
#collection_max_price_eth_df = collection_max_price_df["max_price_for_collection"].div(1700).round(3)

collection_max_price_df


,collection_name,max_price_for_collection
0,Meebits,15000.00
1,CryptoPunks,124457.07
2,BoredApeYachtClub,1024.00
3,MutantApeYachtClub,800.00
4,Otherdeed,851.54
5,Azuki,459.06
6,CloneX,450.00
7,Moonbirds,749.31
8,Doodles,384.03
9,BoredApeKennelClub,626.00


In [13]:
max_price_plot = collection_max_price_df.hvplot.bar(
    height=500,
    width=1000,
    ylabel= " $ USD ",
    xlabel="Collection Name",
    x='collection_name',
    y='max_price_for_collection',
    title="Price Paid of most Expensive NFT by Top 10 Collection",
    rot=90,
    color='orange'
).opts(yformatter='%.0f')

max_price_plot


:Bars   [collection_name]   (max_price_for_collection)

In [14]:
collection_rarity_score_df = df.drop(columns=['contract_id', 'address', 'token_id', 'id_num', 'token_name', 'ranking', 'average_token_rarity_score_for_collection', 'max_price_for_collection', 'avg_price_for_collection', 'min_price_for_collection', 'total_volume_for_collection', 'total_num_trades_for_collection', 'total_unique_buyers_for_collection'])

collection_rarity_score_df


,collection_name,rarity_score
0,Meebits,274.22
1,CryptoPunks,86.19
2,BoredApeYachtClub,68.92
3,MutantApeYachtClub,54.58
4,Otherdeed,44.76
5,Azuki,142.92
6,CloneX,76.83
7,Moonbirds,138.69
8,Doodles,41.22
9,BoredApeKennelClub,91.06


In [15]:
rarity_score_plot = collection_rarity_score_df.hvplot.bar( 
    height=500,
    width=1000,
    ylabel= " Rarity Score ",
    xlabel="Collection Name",
    x='collection_name',
    title="Rarity Score of Rarest NFT by Top 10 Collection",
    rot=90,
    color='green',
).opts(yformatter='%.0f')

rarity_score_plot


:Bars   [collection_name]   (rarity_score)

In [16]:
rarity_score_plot + max_price_plot


:Layout
   .Bars.I  :Bars   [collection_name]   (rarity_score)
   .Bars.II :Bars   [collection_name]   (max_price_for_collection)